## Tasking completion notebook

This script is meant to be used on tasking orders in status "BEING FULFILLED" with a "Polygon" geometry input type. It is for repetitive tasking orders which can not leverage the existing API Get coverage of order assets end point.

The logic:
 - Find active tasking orders in your account (can fine tune to a specific workspace if needed) using 'Tags'
 - Extract the order id, the input AOI geometry and compute in the input AOI surface area
 - Use the order id to find delivered assets
 - Compute delivered assets coverage area in km2, compute coverage percentage.


### Libraries

In [142]:
import up42
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon
from shapely.ops import unary_union

### Learn - step by step

In [ ]:
# Modify the credentials file to authenticate
 
up42.authenticate('./credential.json')
storage = up42.initialize_storage()

#### Find active tasking orders


Extract geometry and order id from active order using tags.

In [ ]:
# Modify or even remove the tags parameter
# Modify or even remove the limit parameter

orders = storage.get_orders(workspace_orders=False,
                            order_type='TASKING',
                            statuses=['BEING_FULFILLED'],
                            limit=5,
                            tags=['pastis'])

In [ ]:
order_ids_list = [order.order_id for order in orders]
order_ids_list

In [ ]:
order_info = orders[1].info
order_info

In [209]:
tasking_geometry = orders[1].order_details['geometry']

# Create the Polygon from coordinates
polygon = Polygon(tasking_geometry['coordinates'][0])

In [ ]:
order_dataframe = gpd.GeoDataFrame([[order_info['id'], order_info['status'], order_info['tags'], polygon]],
                 columns=['order_id', 'status', 'tags', 'geometry'],
                 crs='EPSG:4326')
order_dataframe

In [ ]:
best_utm_zone = order_dataframe.estimate_utm_crs()
best_utm_zone

In [ ]:
order_dataframe_utm = order_dataframe.to_crs(best_utm_zone)
order_dataframe_utm

In [ ]:
order_dataframe_utm_area = order_dataframe_utm.area/1000000
order_dataframe_utm['aoi_area_km'] = round(order_dataframe_utm_area,2)
order_dataframe_utm

#### Delivered data in storage

Find delivered data using the order id

In [ ]:
order_dataframe_utm.iloc[0].order_id

In [ ]:
# check delivery coverage in storage/data management
assets = storage.get_assets(search=order_dataframe_utm.iloc[0].order_id)

In [219]:
delivered_assets_geom_list = [Polygon(asset.stac_items[0].geometry['coordinates'][0]) for asset in assets]
delivered_assets_geom_list

In [ ]:
merged_delivered_assets_geoms = unary_union(delivered_assets_geom_list)
merged_delivered_assets_geoms

In [ ]:
delivered_assets_dt = gpd.GeoDataFrame(geometry=[merged_delivered_assets_geoms], crs='EPSG:4326')
delivered_assets_dt.estimate_utm_crs()
delivered_assets_dt_utm = delivered_assets_dt.to_crs(best_utm_zone)
delivered_assets_dt_utm

In [ ]:
order_summary = order_dataframe_utm
order_summary['delivered_area_km'] = round(delivered_assets_dt_utm.area/1000000, 2)
order_summary['delivered_%'] = round(order_summary['delivered_area_km']*100/order_summary['aoi_area_km'], 2)
order_summary

### Automate

In [ ]:
# Authenticate
up42.authenticate('./credential.json')
storage = up42.initialize_storage()

# Find orders - MODIFY or even remove the tags and the limit parameter
orders = storage.get_orders(workspace_orders=False,
                            order_type='TASKING',
                            statuses=['BEING_FULFILLED'],
                            limit=5,
                            tags=['pastis', 'cs_tag'])


# Build order id list
order_ids_list = [order.order_id for order in orders]

all_order_summaries = gpd.GeoDataFrame()

for num, order_id in enumerate(order_ids_list):

    print(f'Working on order {order_id}')

    # Extract order information
    order_info = orders[num].info

    # Extract the AOI for this order
    tasking_geometry = orders[num].order_details['geometry']

    # Create Polygon from coordinates
    polygon = Polygon(tasking_geometry['coordinates'][0])

    # Create dataframe for the order
    order_dataframe = gpd.GeoDataFrame([[order_info['id'], order_info['status'], order_info['tags'], polygon]],
                    columns=['order_id', 'status', 'tags', 'geometry'],
                    crs='EPSG:4326')

    # Estimate the best UTM zone
    best_utm_zone = order_dataframe.estimate_utm_crs()

    # Convert dataframe to UTM
    order_dataframe_utm = order_dataframe.to_crs(best_utm_zone)

    # Compute the tasking aoi area size
    order_dataframe_utm_area = order_dataframe_utm.area/1000000
    order_dataframe_utm['aoi_area_km'] = round(order_dataframe_utm_area,2)

    # Find delivered data/assets in storage that correspond to this order
    print(' - Searching assets for order...')
    assets = storage.get_assets(search=order_id)
    
    if len(assets) > 0:

        # Place the assets geometries into a list and merge then together
        delivered_assets_geom_list = [Polygon(asset.stac_items[0].geometry['coordinates'][0]) for asset in assets]
        merged_delivered_assets_geoms = unary_union(delivered_assets_geom_list)

        # Create delivered assets dataframe and convert to UTM
        delivered_assets_dt = gpd.GeoDataFrame(geometry=[merged_delivered_assets_geoms], crs='EPSG:4326')
        delivered_assets_dt.estimate_utm_crs()
        delivered_assets_dt_utm = delivered_assets_dt.to_crs(best_utm_zone)

        # Create an order summary dataframe with coverage percentage for the order
        order_summary = order_dataframe_utm
        order_summary['delivered_area_km'] = round(delivered_assets_dt_utm.area/1000000, 2)
        order_summary['delivered_%'] = round(order_summary['delivered_area_km']*100/order_summary['aoi_area_km'], 2)
        completion_percentage = order_summary['delivered_%'].values[0]
        print(f' - {completion_percentage}% complete')

        all_order_summaries = pd.concat([all_order_summaries, order_summary.to_crs('EPSG:4326')], ignore_index=True)
    
    else:
        print(f' - No assets delivered yet for order: {order_id}')
        continue

all_order_summaries